# Tarea 2

- Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.
- Su función debe poder recibir la semilla.
- Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada.
- Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada.

In [1]:
from finrisk import QC_Financial_3 as Qcf
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

#from modules
import auxiliary as aux

from numpy import random as rnd
import plotly.express as px
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

## Función Curva Cero Cupón 

### Curva Cero Cupón en Tiempo Continuo

In [2]:
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
curva['t'] = curva['plazo'] / 365.0
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)
zcurva = interp1d(curva['t'],
                  curva['rate'],
                  kind = 'cubic',
                  fill_value = "extrapolate")

#Nota: t es cuando lo quiero calcular
#      T es el plazo del bono

### Tasas y Derivadas Curva Cero Cupón 

In [3]:
#TASAS Y DERIVADAS DE CURVA CERO CUPON EN FUNCIÓN DEL PLAZO t 
#se calcula con diferencias finitas por la derecha
def zrate(t: float) -> float:
    return zcurva(t)

def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta

def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

### Tasas y Derivadas Forward

In [4]:
#TASAS Y DERIVADA FORWARD EN FUNCION DE PLAZO t
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [5]:
# ELIGIENDO VALORES PREESTABLECIDOS DE SIGMA Y GAMMA
sigma = .015
gamma = .5
# Proxy de la tasa instantánea (corta) r(t) es una tasa entre t y t + dt (donde dt es un intervalo infitesimal),
# para invertir a un intante de tiempo despues
r0 = zrate(0)
# PLAZOS
t = 1
T = 2
# Semilla
seed = 3879

## Función $\theta$

In [6]:
def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

## Simulación MC Tasa Corta

In [7]:
def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):
    """
    Simulación de montecarlo
    
    params:
    - gamma: intensidad de reversión del modelo HW
    - sigma: volatilidad
    - Theta: función theta del modelo HW
    - r0: tasa instantánea
    - num_sim: número simulaciones
    - num_step: número pasas cada simulación
    """
    dt = 1 / 264.0
    num_steps += 1 
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(0, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return tiempo, sim

# Bono Cupón Cero

### Simulaciones MC y Mercado

In [8]:
#VALORES PRESENTES (Precio bono cero cupon)
num_sim = 1000
num_steps = 263 #Un día antes de T
dt = 1 / 264.0 #Saltos Diarios
print(f"num_steps: {num_steps}")
df = 0

tiempo, s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
for sim in s:
     df += math.exp(-dt * np.sum(sim))
    
# MC Bono Cupón Cero en t 
ez = df / num_sim #Promedio simulaciones
print(f"ez: {ez: .8%}") 

# Bono Cupón Cero de Mercado en t 
z_curva = math.exp(-zrate(t) * t)
print(f"z_curva: {z_curva: .8%}") 

num_steps: 263
ez:  99.93976321%
z_curva:  99.92978688%


### Error Relativo Bono Cero Cupón

In [9]:
error=(ez-z_curva)/z_curva
print(f"Error Relativo: {error*100: .8%}")

Error Relativo:  0.99833356%


**AD:** Me hubiera gustado que compararan más plazos.

# Valorizar un Call o Put de un Bono Cupón Cero

## Simulación de Montecarlo

### Tasa Corta

In [10]:
num_sim = 1000
num_steps = 263

#RESULTADOS SIMULACIÓN
tiempo, s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
last_rates = [sim[-1] for sim in s] #Último valor cada simulación 

### Precio Subyacente (Bono Cupón Cero)

In [11]:
#Función A(t) del Modelo HW
def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    Calcula el valor de la función A(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - zrate: curva cero cupón
    - fwd: Tasa forward
    - gamma: intensidad de reversión del modelo HW
    - sigma: volatilidad
    - t: cuando quiero calcular
    - T: plazo del bono

    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [12]:
#Función B(t) del Modelo HW
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t: cuando quiero calcular
    - T: plazo del bono
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [13]:
#Bono Cero Cupón 
def zero_hw(r: float,
            gamma: float,
            sigma: float,
            zrate: float,
            fwd: float,
            t: float,
            T: float) -> float:
    """
    Calculo Bono Cupón Cero
    params:
    - r: tasa simulada
    - gamma: intensidad de reversión del modelo HW
    - sigma: volatilidad
    - zrate: curva cero cupón
    - fwd: Tasa forward
    - t: cuando quiero calcular
    - T: plazo del bono
    """
 
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

In [14]:
#PRECIOS BONO CERO CUPÓN EN t DE PLAZO T A PARTIR DE 1000 TASAS CORTAS SIMULADAS
precios = [zero_hw(r, gamma, sigma, zrate, fwd, t, T) for r in last_rates]
P = pd.DataFrame()
P["PRECIOS"]=precios
#P

#no esta en valor presente por que para esto debo calcular el valor esperado 
#del valor presente y necesito el recorrido de cada simulación

### Payoff Opción

In [15]:
def Vanilla_Payoff(z: float, strike: float, tipo) -> float:
    """
    Cálculo payoff de una call o put 
    
   - z: es el precio del bono cupón cero al vencimiento
   - strike: es el strike de la opción
   - Tipo: es el tipo de opción 
   """
    eps=1
    
    if tipo == "Call":
        eps = 1 
        
    elif tipo == "Put":
        eps = -1
        
    else:
        print("Solo se puede escribir Call o Put como variable tipo")
        
    return max(max(eps*(z - strike), .000), .000)

In [16]:
#Elegir Strikes para Call y Put, respectivamente
strikes_C = [0.3 ,0.7, 0.8, 1.01]
strikes_P = [0.98, 1, 1.2, 1.3, 5]

In [17]:
payoffs_C = {strike:[Vanilla_Payoff(z, strike, 'Call') for z in precios] for strike in strikes_C}

payoffs_P = {strike:[Vanilla_Payoff(z, strike, 'Put') for z in precios] for strike in strikes_P}

### Valores Presentes

In [18]:
#FACTORES DE DESCUENTO
df = np.array(math.exp(-dt * np.sum(s[0])))

for sim in s[1:]:
    df=np.append(df, math.exp(-dt * np.sum(sim)))
                                              

In [19]:
# CALL
Vp_C = {strike:[np.average(np.multiply(df,payoffs_C[strike]))] for strike in strikes_C}
print(Vp_C)
# PUT
Vp_P = {strike:[np.average(np.multiply(df,payoffs_P[strike]))] for strike in strikes_P}
print(Vp_P)


{0.3: [0.6993457458026102], 0.7: [0.2995866929667216], 0.8: [0.1996469297577493], 1.01: [0.0007456031750802969]}
{0.98: [8.049604527382468e-05], 1: [0.00392088008031346], 1.2: [0.20011212307813925], 1.3: [0.3000518862871115], 5: [3.997823125019082]}


In [20]:
#A MANO (Solo funciona con un strike en la lista)
vp_C=0

for strike in strikes_C:
    for i in range(num_sim):
         vp_C += df[i]*payoffs_C[strike][i]
VP_C=vp_C/num_sim
print(VP_C)

vp_P=0

for strike in strikes_P:
    for i in range(num_sim):
         vp_P += df[i]*payoffs_P[strike][i]
VP_P=vp_P/num_sim
print(VP_P)

1.1993249717021608
4.501988510509916


## Valor Teórico Opción

In [21]:
#DEFINIR PLAZOS
T0 = 1
TB = 2

In [22]:
#Comprobación
precio_T0=zero_hw(zrate(0), gamma, sigma, zrate, fwd, 0, T0)
precio_TB=zero_hw(zrate(0), gamma, sigma, zrate, fwd, 0, TB)
print(f"To: {precio_T0: .8}")
print(f"To curva: {math.exp(-zrate(T0) * T0): .8}")
print(f"TB: {precio_TB: .8}")
print(f"TB curva: {math.exp(-zrate(TB) * TB): .8}")

To:  0.99929787
To curva:  0.99929787
TB:  0.99886549
TB curva:  0.99886549


### Payoff Opción 

In [23]:
def Opcion_BS(zrate: float, gamma: float, sigma: float, strike: float, T0: float, TB: float, tipo)-> float:

    #Precios Bono Cupón Cero
    precio_T0=math.exp(-zrate(T0) * T0)
    precio_TB=math.exp(-zrate(TB) * TB)
    
    #Opciones
    eps=1
    
    if tipo == "Call":
        eps = 1 
        
    elif tipo == "Put":
        eps = -1
        
    else:
        print("Solo se puede escribir Call o Put como variable tipo")
   
    #Black Scholes
    Sz = b_hw(gamma, T0, TB)*math.sqrt(sigma**2/(2*gamma)*(1-math.exp(-2*gamma*T0)))
    d1 = (math.log(precio_TB/(strike*precio_T0))+0.5*Sz**2)/Sz
    d2 = d1-Sz
    N1 = norm.cdf(eps*d1)
    N2 = norm.cdf(eps*d2)
        
    return eps*precio_T0*((precio_TB/precio_T0)*N1-strike*N2)

### Valor Presente

In [24]:
# CALL
Vp_BS_C = {strike:[Opcion_BS(zrate, gamma, sigma, strike, T0, TB, 'Call')] for strike in strikes_C}
print(Vp_BS_C)

Vp_BS_P = {strike:[Opcion_BS(zrate, gamma, sigma, strike, T0, TB, 'Put')] for strike in strikes_P}
print(Vp_BS_P)

{0.3: [0.6990761266039472], 0.7: [0.29935697907219233], 0.8: [0.1994271921892535], 1.01: [0.0006385183229155442]}
{0.98: [5.8960634580996594e-05], 1: [0.003960769916741911], 1.2: [0.20029195534250138], 1.3: [0.30022174222544024], 5: [3.9976238568941733]}


### Test

In [26]:
import hull_white as hw

In [31]:
check_call = {strike: hw.zcb_call_put(
    hw.CallPut.CALL,
    strike,
    T0,
    TB,
    zrate(0),
    math.exp(-zrate(T0) * T0),
    math.exp(-zrate(TB) * TB),
    gamma,
    sigma) for strike in strikes_C}

In [32]:
check_call

{0.3: 0.6990761266039472,
 0.7: 0.2993569790721923,
 0.8: 0.19942719218925353,
 1.01: 0.00063851832291556}

In [33]:
check_put = {strike: hw.zcb_call_put(
    hw.CallPut.PUT,
    strike,
    T0,
    TB,
    zrate(0),
    math.exp(-zrate(T0) * T0),
    math.exp(-zrate(TB) * TB),
    gamma,
    sigma) for strike in strikes_P}

In [34]:
check_put

{0.98: 5.896063458099565e-05,
 1: 0.00396076991674188,
 1.2: 0.20029195534250144,
 1.3: 0.3002217422254402,
 5: 3.9976238568941738}

## Error Relativo

In [25]:
#Los errores estan en porcentaje
Error_C = {strike:[(np.array(Vp_C[strike])-np.array(Vp_BS_C[strike]))*100/np.array(Vp_BS_C[strike])] for strike in strikes_C}
print(Error_C)

Error_P = {strike:[(np.array(Vp_P[strike])-np.array(Vp_BS_P[strike]))*100/np.array(Vp_BS_P[strike])] for strike in strikes_P}
print(Error_P)


{0.3: [array([0.03856793])], 0.7: [array([0.07673577])], 0.8: [array([0.11018436])], 1.01: [array([16.77083465])]}
{0.98: [array([36.525066])], 1: [array([-1.00712329])], 1.2: [array([-0.08978507])], 1.3: [array([-0.05657683])], 5: [array([0.00498466])]}


**AD:** Muy poco explicado, es difícil seguir el razonamiento. 